In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
model = SentenceTransformer("bert-base-uncased")

No sentence-transformers model found with name bert-base-uncased. Creating a new one with MEAN pooling.


In [3]:
df = pd.read_csv("data/spotify_millsongdata_w_genres.csv")

In [4]:
song_embeddings = model.encode(df['song'].values, show_progress_bar=True)
lyrics_embeddings = model.encode(df['cleaned_text'].values, show_progress_bar=True)
genres_embeddings = model.encode(df['genres'].values, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
embeddings = []
for i in range(len(song_embeddings)):
    embeddings.append((song_embeddings[i] + lyrics_embeddings[i] + genres_embeddings[i]) / 3)

In [6]:
cos_sim = util.cos_sim(embeddings, embeddings)

c:\Users\berzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\util.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  a = torch.tensor(a)


In [7]:
cos_sim.shape

torch.Size([100, 100])

In [12]:
song_index = 0

In [13]:
similar_songs = np.flip(np.argsort(cos_sim[song_index].numpy()))[1:10]

In [16]:
df.loc[song_index]

artist                                                       ABBA
song                                        Ahe's My Kind Of Girl
cleaned_text    look at her face , it 's a wonderful face and ...
genres                                                   Rock,Pop
Name: 0, dtype: object

In [15]:
odf = df.loc[similar_songs]
odf['similarity'] = cos_sim[song_index].numpy()[similar_songs]
odf

,artist,song,cleaned_text,genres,similarity
34,ABBA,I Am Just A Girl,"i am just a girl one among the others , nothin...","Rock,Pop",0.929272
92,ABBA,That's Me,are you sure you want to hear more what if i a...,"Rock,Pop",0.920342
59,ABBA,Me And I,sometimes when i 'm mad there 's a part of me ...,"Rock,Pop",0.908379
13,ABBA,Does Your Mother Know,"you 're so hot , teasing me so you 're blue bu...","Rock,Pop",0.903128
52,ABBA,Love Has It's Ways,spring in the air the sun was shining still th...,"Rock,Pop",0.901911
43,ABBA,I'm A Marionette,you 're so free that 's what everybody 's tell...,"Rock,Pop",0.899278
81,ABBA,Should I Laugh Or Cry,frida he stands towering over me beside my bed...,"Rock,Pop",0.895975
50,ABBA,Lay All Your Love On Me,i was n't jealous before we met now every woma...,"Rock,Pop",0.892506
8,ABBA,Crazy World,i was out with the morning sun could n't sleep...,"Pop,Rock",0.892207
